In [13]:
'''
Jessie Covington

the shuffle fn takes a sequence of integers and implements a riffle shuffle
it returns a sequence of shuffled integers

input of shuffle is a list or array
(if input is not an np.array we'll change it to one - just because I prefer np.arrays to work with)

The top card in the deck is the first integer of the sequence and the card at the bottom of the deck is the last number
in the sequence

'''

import numpy as np
from random import randint

def shuffle(D):
    ## input: sequence of integers D that we want to shuffled, list or array
    ## output: sequence of shuffled integers shuffled_deck, np.array
    
    deck = np.array(D)  
    ## if not already numpy array, convert to one - if this is already a np.array, will not change
    
    n = len(deck) ## how many cards are in our deck?
    
    shuffled_deck = np.zeros(n) ## allocate new deck for our shuffled cards
    ## so we can maintain our original sequence
    
    n_right_orig = get_random_number_for_right_deck(n) 
    ## print("How many cards in RHD: ", n_right_orig)
    
    ## want to keep track of how many cards we start with in our right deck - will use later
    
    ## n_left = number of total cards - number of cards in right deck
    n_right = n_right_orig
    n_left = n - n_right_orig
    
    ## We need to drop a total of num_cards cards onto the top of our shuffled_deck
    ## Dropping from the BOTTOM of the subdecks !!
    ## cards from top of original deck become right subdeck - like if I was shuffling a deck of cards for real 
    ## so beginning of sequence
    ## cards at end of sequence are bottom of deck
    
    back = n-1
    for i in range(n):
        
        drop_right = should_drop_from_right_deck(n_left, n_right)
        ## print("n_left %s, n_right %s" %(n_left, n_right)) --> was making sure these actually decreased
        
        if drop_right == True:
            ## print("RIGHT")
            ## print(back-i)
            shuffled_deck[back-i] = deck[n_right-1] ## working from bottom of right deck
            n_right -= 1 # one less card in our right subdeck since it's on shuffled deck now
            
            ## need to be dropping into the back of the new array too!! 
            ## what is dropped first ends up at the bottom of the deck as more is dropped
            ## print("\nCard that SHOULD be dropped: ", deck[n_right-1])
            ## print("Card dropped: ", shuffled_deck[back-i])
            
        else:
            ## print("LEFT")
            ## print(back-i)
            next_card = n_right_orig + n_left - 1 # index of next card to drop from left subdeck
            shuffled_deck[back-i] = deck[next_card]
            n_left -= 1 ## we now have one less card in our left subdeck
            ## print("\nCard that SHOULD be dropped: ", deck[next_card])
            ## print("Card dropped: ", shuffled_deck[back-i])
            
        
    return shuffled_deck

##
## below are the functions used in shuffle to determine size of subdecks and from which subdeck to drop
##

'''

Should this one be based on binomial distribution?? I think so... 
Every time (in real life) that we cut a deck of cards, we get roughly an equal number,
but will rarely get exactly half, the cards in each subdeck

Update: yes, see https://www.stat.berkeley.edu/~aldous/150/Lectures/lecture_37_shuffling.pdf
http://tcs.nju.edu.cn/wiki/index.php/%E9%9A%8F%E6%9C%BA%E7%AE%97%E6%B3%95_(Fall_2011)/Card_Shuffling
'''

def get_random_number_for_right_deck(n): 
    
    ## input: n total number of cards and the highest number that can be returned by this fn  
    ## output: int, how many cards should be split into the right subdeck
    
    ## when shuffling a real deck of cards, we split the deck roughly in half - want the same to happen here, so:
    ## should be (n, 0.5) whre n is the total number of cards
    p = 0.5 ## 'probability of success' - in gsr model should be 1/2
    
    num_right = np.random.binomial(n, p)
    ## print(num_right)
    return num_right

'''
Should be
Based on probability of card actually coming from right deck: R / (R+L) 
where L and R are num cards in left and right deck respectively
so:
n_right / (n_left + n_right)

Bernoulli distribution, yeah? 
Update: Yes. From Wikipedia: 
"the probability distribution of any single experiment that asks a yes–no question; 
the question results in a boolean-valued outcome, a single bit of information whose 
value is success/yes/true/one with probability p and failure/no/false/zero with probability q"
'''

def should_drop_from_right_deck(n_left, n_right): 
    ## input: n_left, n_right ints - how many cards are in the left and right subdecks, respectively
    ## output: Boolean - tells us whether we should drop a card from the right subdeck (True) or left (False)
    
## or if the subdecks are empty
    ## determine if we should drop a card from the right or left subdeck onto the shuffled pile
    
    if (n_left > 0 and n_right > 0):
        ## num = randint(0, 1)  ## remember to change this one!!
        ## bc this is not correct way to do it but it's giving us a random-ish subdeck for the time being
        ## so I can at least see if shuffling is working
        
        p = n_right / (n_right + n_left) ## probability that a card comes from right deck
        ## q = 1 - p ## probability that card comes from left deck
        
        num = np.random.binomial(1, p) ## bernoulli \equiv binomial with just a single trial
    
        ## print("\n n_right %s and n_left %s" %(n_right, n_left))
        ## print("p is %s and num %s" %(p, num))
        
        drop_right_bool = num == 1
        return drop_right_bool
    
    ## the rest is fine though
    elif (n_left == 0 and n_right > 0): ## left subdeck empty so we have to drop from the right deck
        ## print("\n n_right %s and n_left %s" %(n_right, n_left))
        return True
    
    elif (n_left > 0 and n_right == 0): ## right subdeck empty so we have to drop from the left deck
        ## print("\n n_right %s and n_left %s" %(n_right, n_left))
        return False 
    
    else: ## n_left and n_right = 0, empty subdecks
        ## print("\n n_right %s and n_left %s" %(n_right, n_left))
        raise ValueError("The subdecks are empty") ## can't drop from empty deck, n_right and/or n_left have to be >0

In [19]:
## now the decks we'll actually be considering:
D13 = list(range(0, 13))
D26 = list(range(0, 26))
D52 = list(range(0, 52))
D104 = list(range(0, 104))

deck_list = np.array([D13, D26, D52, D104]) ## keep all our original decks in one place together
n = len(deck_list)  ## this will be used throughout - number of original and shuffled decks

num_shuffles = 7 ## how many shuffles do we want to try?

### so we can keep track of what the deck looks like after each indivisdual shuffle:
deck1_shuffles = np.zeros((num_shuffles+1, len(D13)))
deck2_shuffles = np.zeros((num_shuffles+1, len(D26)))
deck3_shuffles = np.zeros((num_shuffles+1, len(D52)))
deck4_shuffles = np.zeros((num_shuffles+1, len(D104)))

deck1_shuffles[0] = D13
deck2_shuffles[0] = D26
deck3_shuffles[0] = D52
deck4_shuffles[0] = D104

In [20]:
print("Let's consider where the original top and bottom cards of the unshuffled deck end up after 7 shuffles.")
print()

top_card_num_arr = np.zeros(n) ## keep track of our first to see where it ends up - in this case they're all 0
## but might have a case where it's not later, so.
bottom_card_num_arr = np.zeros(n) ## keep track of last element so we can see where it ends up

init_top_card_index = np.zeros(n) ## so we can compare later - all will remain zero since first index
init_bottom_card_index = np.zeros(n)

new_top_card_index = np.zeros(n) ## store the index of where the original top card ends up
new_bottom_card_index = np.zeros(n) ## '' '' bottom '' ''

## so we can keep our original decks if we need to use them later and bc otherwise Si won't be recognized
S1 = D13
S2 = D26
S3 = D52
S4 = D104

## shuffle each deck however many times we wanted to shuffle (declared above)
for i in range(num_shuffles):
    S1 = shuffle(S1)
    S2 = shuffle(S2)
    S3 = shuffle(S3)
    S4 = shuffle(S4)
    
    deck1_shuffles[i+1] = S1
    deck2_shuffles[i+1] = S2
    deck3_shuffles[i+1] = S3
    deck4_shuffles[i+1] = S4

## putting them in a list to make easier and shorter to access
shuffled_deck_list = [S1, S2, S3, S4]

for i in range(n):
    ## store the first and last elements(top and bottom cards) of each deck
    top_card_num_arr[i] = deck_list[0][0]
    bottom_card_num_arr[i] = deck_list[i][-1]
    
    ## find indices of bottom card of deck (doing this way in case we change our deck sizes later)
    init_bottom_card_index[i] = len(deck_list[i]) - 1

    ## store the index of where the first and last cards moved
    new_top_card_index[i] = np.where(shuffled_deck_list[i] == top_card_num_arr[i])[0][0]
    new_bottom_card_index[i] = np.where(shuffled_deck_list[i] == bottom_card_num_arr[i])[0][0]
    
    ## print(bottom_card_num_arr)

    ## shuffled_deck_list[i] = shuffled_deck_list[i].tolist()
    print("The shuffled deck %s is: \n %s \n" %(i+1, shuffled_deck_list[i]) )

## print(new_top_card_index, new_bottom_card_index)

Let's consider where the original top and bottom cards of the unshuffled deck end up after 7 shuffles.

The shuffled deck 1 is: 
 [  7.   8.   3.   4.  11.  12.   5.   6.  10.   2.   9.   0.   1.] 

The shuffled deck 2 is: 
 [ 20.   3.   9.  11.   5.  23.   0.  19.   4.   8.  12.  18.  10.  25.   7.
  14.   1.  16.  17.   6.  21.  24.  22.  13.   2.  15.] 

The shuffled deck 3 is: 
 [ 48.  49.  17.  40.  27.  38.  47.  25.  23.  35.  45.  29.   1.  39.  10.
  50.   4.  24.   3.  18.  12.  36.  20.  31.  22.  14.  11.  44.  41.  32.
   2.  13.   7.   9.   6.  26.  28.  30.  21.  51.  19.  33.  15.  34.   0.
  37.   8.  46.   5.  42.  16.  43.] 

The shuffled deck 4 is: 
 [  63.   89.   57.   20.   64.   92.   43.    0.   29.   55.   84.   46.
   73.   18.   93.   35.   99.   70.    4.   32.   60.   71.   62.   91.
   65.  100.   24.  101.   15.   25.   80.   67.  102.   97.   37.   95.
   17.   41.   26.   42.    1.    9.   11.   30.   96.   53.   77.   13.
   23.   47.   19.    7.   74